In [1]:
import tensorflow as tf
import numpy as np
import time
import os

In [2]:
from keras.datasets import cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()

img_row, img_col = img_train.shape[1], img_train.shape[2]
#img_depth = 1
img_depth = img_train.shape[3]

#img_train = img_train.reshape(img_train.shape[0], img_row, img_col, img_depth)
#img_test = img_test.reshape(img_test.shape[0], img_row, img_col, img_depth)
label_train = np.squeeze(label_train, axis=-1)
label_test = np.squeeze(label_test, axis=-1)
img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

C:\Users\Hyoje\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def conv_layer(inputs, filters, name):
    with tf.variable_scope('{}/conv_layer'.format(model)):
        conv = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=[3, 3],
                                strides=[1, 1], padding='same', name=name)
    return conv

def max_pool(inputs, name):
    with tf.variable_scope('{}/conv_layer'.format(model)):
        pool = tf.layers.max_pooling2d(inputs=inputs, strides=[2, 2], pool_size=[2, 2],
                                       padding='same', name=name)
    return pool

In [4]:
def get_accuracy(sess, logits, img, label, batch_size):    
    if len(img) < batch_size:
        logit = sess.run(logits, feed_dict={X: img, keep_prob: 1.0})
    else:
        total_batch = int(len(img) / batch_size) + 1
        num_label = int(logits.shape[-1])
        logit = np.array([], dtype=np.int64).reshape(0, num_label)
        idxs = range(len(img))
        for i in range(total_batch):
            if i < (total_batch - 1):
                idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
            else:
                idxs_i = idxs[i * batch_size : len(img)]
            feed_dict = {X: img[idxs_i], keep_prob: 1.0}
            logit = np.concatenate((logit, sess.run(logits, feed_dict=feed_dict)), axis=0)

    correct_pred = np.equal(np.argmax(logit, 1), label).astype(np.float32)
    accuracy = np.mean(correct_pred)
    
    return accuracy, logit, correct_pred

In [5]:
# hyper prameter
model = 'VGGNet16'
batch_size = 128
epochs = 20
CHECK_POINT_DIR = './save_model_{}'.format(model)

In [6]:
X = tf.placeholder(tf.float32, [None, img_row, img_col, img_depth])
Y = tf.placeholder(tf.int64, [None])
keep_prob = tf.placeholder(tf.float32)

In [7]:
# VGG16

# convolution layers

conv1 = conv_layer(inputs=X, filters=64, name='conv1')
conv2 = conv_layer(inputs=conv1, filters=64, name='conv2')
max1 = max_pool(inputs=conv2, name='max1')

conv3 = conv_layer(inputs=max1, filters=128, name='conv3')
conv4 = conv_layer(inputs=conv3, filters=128, name='conv4')
max2 = max_pool(inputs=conv4, name='max2')

conv5 = conv_layer(inputs=max2, filters=256, name='conv5')
conv6 = conv_layer(inputs=conv5, filters=256, name='conv6')
conv7 = conv_layer(inputs=conv6, filters=256, name='conv7')
max3 = max_pool(inputs=conv7, name='max3')

conv8 = conv_layer(inputs=max3, filters=512, name='conv8')
conv9 = conv_layer(inputs=conv8, filters=512, name='conv9')
conv10 = conv_layer(inputs=conv9, filters=512, name='conv10')
max4 = max_pool(inputs=conv10, name='max4')

conv11 = conv_layer(inputs=max4, filters=512, name='conv11')
conv12 = conv_layer(inputs=conv11, filters=512, name='conv12')
conv13 = conv_layer(inputs=conv12, filters=512, name='conv13')
max5 = max_pool(inputs=conv13, name='max5')

conv = max5

In [8]:
# fully connected layers

flat_num = int(conv.shape[1] * conv.shape[2] * conv.shape[3])
flat = tf.reshape(conv, [-1, flat_num])

with tf.variable_scope('{}/fc_layer'.format(model)):
    dense1 = tf.layers.dense(inputs=flat, units=4096, 
                            activation=tf.nn.relu, name='dense1')
    drop1 = tf.nn.dropout(x=dense1, keep_prob=keep_prob)
    dense2 = tf.layers.dense(inputs=drop1, units=4096, 
                            activation=tf.nn.relu, name='dense2')
    drop2 = tf.nn.dropout(x=dense2, keep_prob=keep_prob)
    logits = tf.layers.dense(inputs=drop2, units=10, name='logit')        

In [9]:
np.sum([np.prod(v.shape) for v in tf.trainable_variables()])

Dimension(33638218)

In [10]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
config=tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allocator_type='BFC'
config.log_device_placement=False
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [12]:
# Restore and Save

saver = tf.train.Saver()
checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)

if checkpoint and checkpoint.model_checkpoint_path:
    epoch_num = int(checkpoint.model_checkpoint_path.split('-')[-1]) + 1
    max_accuracy, _, _= get_accuracy(sess, logits, img_test, label_test, batch_size)
    try:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    except:
        print("Error on loading old network weights")
else:
    epoch_num = 1
    max_accuracy = 0.0
    print("Could not find old network weights")

Could not find old network weights


In [ ]:
# train

print(time.asctime())
print ('Learning Started.')

total_start = time.time()

for epoch in range(epochs):
    epoch_start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size) + 1
    
    idxs = np.random.permutation(range(len(img_train)))
    for i in range(total_batch):
        if i < (total_batch - 1):        
            idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
        else:
            idxs_i = idxs[i * batch_size : len(img_train)]
            
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i], keep_prob: 0.5}    
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print ('=====================================')
    print ('Epoch : {:02d} , Cost : {:.5f}'.format(epoch + epoch_num, avg_cost))
    train_accuracy, _, _ = get_accuracy(sess, logits, img_train, label_train, batch_size)
    print ('Train Accuracy : {:.5f}'.format(train_accuracy))
    test_accuracy, _, _ = get_accuracy(sess, logits, img_test, label_test, batch_size)
    print ('Test Accuracy : {:.5f}'.format(test_accuracy))
    print ('Elapsed time of an epoch: {:.5f}'.format(time.time() - epoch_start))
    if test_accuracy > max_accuracy:
        max_accuracy = test_accuracy
        print ('===========Saving network with the best accuracy===========')
        if not os.path.exists(CHECK_POINT_DIR):
            os.makedirs(CHECK_POINT_DIR)
        saver.save(sess, CHECK_POINT_DIR + "/model", global_step=epoch+epoch_num)

print ('=====================================')

print ('Final Test Accuracy : {:.5f}'.format(max_accuracy))
print ('Total elapsed time: {:.5f}'.format(time.time() - total_start))

Wed Jan 31 15:15:33 2018
Learning Started.
Epoch : 01 , Cost : 2.85063
Train Accuracy : 0.34646
Test Accuracy : 0.34870
Elapsed time of an epoch: 142.28625
===========Saving network with the best accuracy===========
Epoch : 02 , Cost : 1.59873
Train Accuracy : 0.48180
Test Accuracy : 0.47130
Elapsed time of an epoch: 140.34008
===========Saving network with the best accuracy===========
Epoch : 03 , Cost : 1.33612
Train Accuracy : 0.61064
Test Accuracy : 0.59430
Elapsed time of an epoch: 141.00885
===========Saving network with the best accuracy===========
Epoch : 04 , Cost : 1.13290
Train Accuracy : 0.64296
Test Accuracy : 0.61080
Elapsed time of an epoch: 140.37392
===========Saving network with the best accuracy===========
Epoch : 05 , Cost : 1.01777
Train Accuracy : 0.67684
Test Accuracy : 0.63520
Elapsed time of an epoch: 139.19032
===========Saving network with the best accuracy===========
Epoch : 06 , Cost : 0.95414
Train Accuracy : 0.72600
Test Accuracy : 0.67060
Elapsed time of